#### Parte I - Análisis de la base de hogares y cálculo de pobreza

##### 1. Analisis exploratorio

Comezamos pensando, solo intuitvamente cuales a priori son las variables que podrian ser utilies para predecir la pobreza y perfeccionar las estimaciones el TP2:

Dentro de la varibles en la encuesta de hogares consideramos relevantes las caracterisicas de la vivienda:
* El tipo de vivienda (IV1)
* Cantidad de habitaciones (IV2)
* Material del techo (V4)
* Revestimiento del techo (IV5)
* Tipo de conexion a agua (IV6 y IV7)
* Tener baño (IV8), tipo de baño (IV9) y equipamiento (IV10)
* Tipo de desague (IV11)
* Zona de ubicacion: cercano a basural (IV12_1), zona inundable (IV12_2) y/o villa de emergecia (IV12_3)


Las caractersiticas habitacionales:
* Cantidad de ambientes (II1)
* Cantidad de dormitorios (II2)
* Habitaccion para trabajo (II3), exlusivamente (II6)
* Tienen: cuarto de cocina (II4_1), lavadero (II4_2), garage (II4_3), cuantos de estos se usan para dormir (II5).
* Regimen de tenencia (II7)
* Combustible para cocinar (II8)
* Propiedad del baño (II9)

Variables de tipo de ingresos: V1 a 18. Y variables de trabajo infantil: V19_A y V19_B

Integrantes del hogar: total (IX_Tot) y menores de 10 (IX_men10).

No icluimos las que son mediciones de ingresos al igual que en el TP2.





2. Continuamos descargando la base de hogares y cargandola

In [5]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Definimos el directorio
os.chdir('C:/Users/Usuario\Desktop/MAESTRIA\Big Data/TPs/BigData/TP3')

In [6]:
# Cargamos los datos, menteniendo solo las observaciones para el Gran Buenis Aires y la Ciudad de Buenos Aires:
hogares = pd.read_excel("inputs/usu_hogar_T123.xlsx")
hogares = hogares[(hogares['AGLOMERADO']==32) | (hogares['AGLOMERADO']==33)]
individual = pd.read_excel("inputs/usu_individual_T123.xlsx")
individual = individual[(individual['AGLOMERADO']==32) | (individual['AGLOMERADO']==33)]

3. Unimos la tablas de Individuos y hogares

In [7]:
# Unimos las tablas
# left join individual con hogares por CODUSU
df = pd.merge(individual, hogares, on=['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'REGION', 'MAS_500', 'AGLOMERADO','PONDERA','PONDIH','ITF'], how='left')

4. Herramientas de limpieza de datos

Para la limpieza utilizaremos las herramientas que nos ofrece el modulo `pandas`, para el manejo de DataFrames. A continuación, explicaré las funciones que se están utilizando:

   - la función `describe()` proporciona información resumida sobre la distribución de valores en esa columna. Incluyendo la media, minimo, desviación estánda, cuartiles, etc.

   - `isnull()` es un método de los DataFrames de pandas que devuelve una matriz booleana indicando las ubicaciones de los valores faltantes en el DataFrame.

   - `dropna()` es un método de los DataFrames de pandas que elimina las filas o columnas con valores faltantes.

   - La función `fillna()` se utiliza para rellenar los valores faltantes en la columna.

   - La función `drop()` con `axis=1`, elimina una columna inidicada.

   -  La función `duplicated()` es un método de pandas que devuelve una Serie de valores booleanos que indica si cada fila del DataFrame es una duplicada de una fila anterior. Con `sum()` contamos estos booleanos generado de modo de obtener la cantidad de duplicados.


5. Limpieza de Datos

In [8]:
# Hacemos un cheaqueo rapido de los datos, generando una tabla de estadisticas descriptivas
descripcion = df.describe()

In [9]:
# Chequeamos que no haya duplicados
df.duplicated().sum()

0

In [10]:
#Notese quiebre de NAs en 96% y 81%, no es tan arbitrario el 70%
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_eph = pd.DataFrame({'Columna': df.columns,
                                 'Porcentaje de NAs': percent_missing})

In [12]:

# Eliminimamos la varaible si esta tiene mas de 90% de percent_missing
df = df.dropna(thresh=len(df)*0.9, axis=1)
# Chequeamos nuevamente los missing
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_eph = pd.DataFrame({'Columna': df.columns,
                                 'Porcentaje de NAs': percent_missing})
# Remplazamos los missings en CH08 (cobertura medica) por 9 = Ns/Nc
df['CH08'] = df['CH08'].fillna(9)
# Eliminamos la variable PT47, dado que es una varaible de ingreso y la eliminariamos de todos modos luego
df = df.drop(['P47T'], axis=1)

# De este modo no tenemos mas NAs

In [14]:
# Cheaqueamos que no existan ingresos negativos
df['ITF'].describe()


count    7.619000e+03
mean     1.384079e+05
std      3.349417e+05
min      0.000000e+00
25%      0.000000e+00
50%      7.500000e+04
75%      2.000000e+05
max      1.099000e+07
Name: ITF, dtype: float64

In [15]:
# Cheqqueamos que no haya edades negativas
df['CH06'].describe()


count    7619.000000
mean       37.177845
std        22.738454
min        -1.000000
25%        18.000000
50%        35.000000
75%        55.000000
max        99.000000
Name: CH06, dtype: float64

In [16]:

# Remplazamos los valores negativos por 1 (suponemos que son menores de 1 año)
df['CH06'] = df['CH06'].replace([-1], [1])
df['CH06'].describe()


count    7619.000000
mean       37.190445
std        22.717837
min         1.000000
25%        18.000000
50%        35.000000
75%        55.000000
max        99.000000
Name: CH06, dtype: float64

6. Estadisticas descriptivas

A continuación presentamos las estadisticas descriptivas, de cinco variable de la encuesta de hogares que consideramos relevantes: Total  de integrantes del hogar(IX_Tot), integrantes menores de 10 años (IX_men10), cantidad de habitaciones (IV2), vivienda en zona inundable (IV12_2), y vivienda en villa de emergecia (IV12_3)

In [24]:
# Eliminamos las observaciones de hogares que se duplican por individuo
variables_interes = df.drop_duplicates(subset='CODUSU', keep='first')
# Varaibles de interes
variables_interes = variables_interes[['IX_TOT', 'IX_MEN10', 'IV2', 'IV12_2', 'IV12_3']]

variables_interes.describe()

,IX_TOT,IX_MEN10,IV2,IV12_2,IV12_3
count,2724.000000,2724.000000,2724.00000,2724.000000,2724.000000
mean,2.783407,0.345448,2.91630,1.923642,1.994493
std,1.594635,0.712393,2.15749,0.265619,0.074016
min,1.000000,0.000000,1.00000,1.000000,1.000000
25%,2.000000,0.000000,2.00000,2.000000,2.000000
50%,2.000000,0.000000,3.00000,2.000000,2.000000
75%,4.000000,0.000000,3.00000,2.000000,2.000000
max,13.000000,8.000000,99.00000,2.000000,2.000000


Podemos ver que para los 2742 hogare en la emcuesta, en promedio  hay 3 integrantes por hogar, y que aproximadamente solo uno de cada tres hogares tienen un nicño menor a 10 años. En promedio hay 3 habitaciones por hogar, y solo el 2% de los hogares se encuentran en zonas inundables o en villas de emergencia.

7. Calculo de Adulto Equivalente y Requerimentos del Hogar

In [25]:
# Cargaos la tabla de equivalencia de adultos
adulto_equiv_data = pd.read_excel("inputs/tabla_adulto_equiv.xlsx")

# melt adulto_equiv_data Muejeres y Hombres
adulto_equiv_data = pd.melt(adulto_equiv_data, id_vars=['Edad'], value_name= 'adulto_equiv')

# Change values of variable column to 1 if Hom and 2 if Muj
adulto_equiv_data['variable'] = np.where(adulto_equiv_data['variable']=='Mujeres', 2, 1)
# Rename variable column to Sexo
adulto_equiv_data = adulto_equiv_data.rename(columns={'variable': 'Sexo'})

#Función que lee los valores de edad en números CH06 y me lo impacta en la categoría etarea correspondiende de la nueva tabla.
def rango_edad(edad):
    if edad < 18:
        rangoetareo = str(edad)+" años"
    elif 17 < edad and edad < 30:
        rangoetareo = "18 a 29 años"
    elif 29 < edad and edad < 46:
        rangoetareo = "30 a 45 años"
    elif 45 < edad and edad < 61:
        rangoetareo = "46 a 60 años"
    elif 60 < edad and edad < 76:
        rangoetareo = "61 a 75 años"
    elif edad > 75:
        rangoetareo = "más de 75 años"
 #   elif for n in range(1,18):
  #      rangoetareo = 
        
    else:
        rangoetareo = "ERROR"
    return rangoetareo

#Aplico la función a mi tabla para crear la columna deseada
df['rango_etareo'] = df['CH06'].apply(rango_edad)


#Renombro la columna "edad", igual que la de la otra tabla
df = df.rename(columns={'rango_etareo': 'Edad'})
df = df.rename(columns={'CH04': 'Sexo'})


# Hacemos el merge de las dos tablas
df = df.merge(adulto_equiv_data, on=['Sexo','Edad'], how='left')
#Sumo para las personas de un mismo hogary loguardo como ad_equiv_hogar
df['ad_equiv_hogar'] = df.groupby('CODUSU')['adulto_equiv'].transform('sum')



8. Particiones segun respuesta y ingreso necesario

In [26]:
# Punto 1.3: Particionamos la muestra segun respondieron o no sobre los ingresos:
respondieron = df[df['ITF'] > 0]
respondieron = respondieron.reset_index(drop=True)
norespondieron = df[df['ITF'] <= 0]
norespondieron = norespondieron.reset_index(drop=True)
# Las guaradamos para reutilizarlas en la parte 3
respondieron.to_csv('Inputs/df_respondieron.csv')
norespondieron.to_csv('Inputs/df_norespondieron.csv')

In [27]:

# Punto 1.4: Agregar columna que indica el ingreso necesario del hogar para no ser pobre
respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * 53371.05


9. Generamos una indicadora de persona pobre

In [28]:

# Punto 1.4: Columna que indica si una persona es pobre según el ingreso de su hogar
respondieron.loc[:, 'pobre'] = (respondieron['ingreso_necesario'] > respondieron['ITF']).astype(int)
mean_pobres = respondieron['pobre'].mean()
print(mean_pobres)


0.3502610346464167


10. Calculo de pobreza por hogares (muestra expandida)

In [29]:
# Calaculamos la tasa de pobreza de hogares, expondiendo la muestra con el ponderador de hogares, PONDIH.
# Columna indicadora de pobreza de hogares
respondieron.loc[:, 'pobre_hogar'] = (respondieron['ingreso_necesario'] > respondieron['ITF']).astype(int)

# Filtar por unico CODUSU
respondieron = respondieron.drop_duplicates(subset=['CODUSU'])

# Tasa de pobreza de hogares (sin expandir)
mean_pobres_hogar_temp = respondieron['pobre_hogar'].mean()
print(mean_pobres_hogar_temp)


0.23540372670807452


In [30]:
# Ponderar los hogares pobres
respondieron['pobre_hogar_ponde'] = respondieron['PONDIH'] * respondieron['pobre_hogar']

# Tasa de pobreza de hogares
mean_pobres_hogar = respondieron['pobre_hogar_ponde'].sum() / respondieron['PONDIH'].sum() 
print(mean_pobres_hogar)

0.26332527581832327


Encontramos que la pobreza en nuestro calculo es solo del 26,33% para el Gran Buenos Aires y la Ciudad de Buenos Aires. Mientras que en el TP2, la pobreza era del 31,5% para el Gran Buenos Aires. Lo cual, aunque cercano, es menor a al dato publicado en el informe de indec el cual da un numero de 30.3%.